# GCN 

In [1]:
import numpy as np

In [3]:
A = np.matrix([
    [0,1,0,0],
    [0,0,1,1],
    [0,1,0,0],
    [1,0,1,0],
],dtype=float)

In [5]:
#接下来，我们需要抽取出特征！基于每个节点的索引为其生成两个整数特征，这简化了本文后面手动验证矩阵运算的过程。
X = np.matrix([[i,-i] for i in range(A.shape[0])],dtype=float)
X

matrix([[ 0.,  0.],
        [ 1., -1.],
        [ 2., -2.],
        [ 3., -3.]])

In [6]:
#应用传播规则
# 邻接矩阵A 特征集合为X  应用传播规则后
A*X  # 每个节点的表征(每一行)现在是其邻节点特征的和，换句话说，图卷积层将每个节点表示为其相邻节点的聚合。

matrix([[ 1., -1.],
        [ 5., -5.],
        [ 1., -1.],
        [ 2., -2.]])

## 这里有个问题
- 节点的聚合特征不包含他自己的特征，改表征是相邻节点的特征聚合，因此只有具有自环的节点才会在该聚合中包含自己的特征。
- 度大的节点在其特征表述中具有较大的值，度小的节点将具有较小的值。这可能导致梯度小时和梯度爆炸。(随机梯度下降算法通常被用于信息这类网络，且对每个输入特征规模(或值的范围)都很敏感)

In [7]:
# 增加自环 与单位阵相加
I = np.matrix(np.eye(A.shape[0]))
I

matrix([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

In [8]:
A_hat = A+I
A_hat*X #现在每个节点在对相邻节点的特征求和过程中也会囊括自己的特征

matrix([[ 1., -1.],
        [ 6., -6.],
        [ 3., -3.],
        [ 5., -5.]])

## 对特征表征进行归一化处理
通过将邻接矩阵与度矩阵D的逆相乘，对其进行变换，从而通过节点的度对特征表示征进行归一化
$ f(X,A)= D^{-1}AX$

In [9]:
D = np.array(np.sum(A,axis=0))[0]
D = np.matrix(np.diag(D))
D

matrix([[1., 0., 0., 0.],
        [0., 2., 0., 0.],
        [0., 0., 2., 0.],
        [0., 0., 0., 1.]])

In [10]:
D**-1*A

matrix([[0. , 1. , 0. , 0. ],
        [0. , 0. , 0.5, 0.5],
        [0. , 0.5, 0. , 0. ],
        [1. , 0. , 1. , 0. ]])

In [11]:
D**-1*A*X

matrix([[ 1. , -1. ],
        [ 2.5, -2.5],
        [ 0.5, -0.5],
        [ 2. , -2. ]])

In [13]:
W = np.matrix([
    [1,-1],
    [-1,1]
])

D_hat= D+I
D_hat

matrix([[2., 0., 0., 0.],
        [0., 3., 0., 0.],
        [0., 0., 3., 0.],
        [0., 0., 0., 2.]])

In [14]:
D_hat**-1 *A_hat*X*W

matrix([[ 1., -1.],
        [ 4., -4.],
        [ 2., -2.],
        [ 5., -5.]])

In [15]:
# 如果想减小输出特征表征维度，可以减小权重矩阵W的规模
W = np.matrix([
    [1],
    [-1]
])
D_hat**-1 *A_hat*X*W

matrix([[1.],
        [4.],
        [2.],
        [5.]])

In [17]:
W = np.matrix([
    [1,-1],
    [-1,1]
])
H = D_hat**-1 *A_hat*X*W

In [18]:
(H > 0) * 1

matrix([[1, 0],
        [1, 0],
        [1, 0],
        [1, 0]])